In [6]:
import src.multiagent_mpc as mpc
import src.traffic_world as tw
import src.car_plotting_multiple as cmplot
import glob
import argparse
import json
import pickle
import os
import numpy as np

# Arguments
camera_speed = -1
shape = 'image'
svo_colors = 1
n_workers = 8
fps = 16

LOG_DIRECTORY = '/home/nbuckman/mpc_results/02_17_txe/61fc-3c7g-20210218-142624/61fc-3c7g-20210218-142624_00023/'

VEHICLE_PATH = LOG_DIRECTORY + 'ambulance.p'
OTHER_VEHICLE_PATH = LOG_DIRECTORY + 'other_vehicles.p'
TRAJ_PATH = LOG_DIRECTORY + 'trajectories.npy'
PARAMS_PATH = LOG_DIRECTORY + 'params.json'
WORLD_PATH = LOG_DIRECTORY + 'world.p'


trajectory = np.load(open(TRAJ_PATH,'rb'))
xamb = trajectory[0, :, :]
xothers = trajectory[1:, :, :]

log_string = LOG_DIRECTORY.split('/')[-2]
VIDEO_OUT_PATH = LOG_DIRECTORY + log_string + '.mp4'


# generate pictures for animation


In [7]:

# Load data
with open(PARAMS_PATH) as fp:
    params = json.load(fp)

response_vehicle = pickle.load(open(VEHICLE_PATH,'rb'))
other_vehicles = pickle.load(open(OTHER_VEHICLE_PATH,'rb')) #TODO:  save all the vehicles in one file
world = pickle.load(open(WORLD_PATH,'rb'))


# Prep a temp image directory


if camera_speed == -1:
    camera_speed = np.mean(xamb[4,:])

if svo_colors == 1:
    svo = []
    for veh in other_vehicles:
        if hasattr(veh, "theta_ij"):
            svo += [veh.theta_ij[-1]]
        else:
            svo += [veh.theta_iamb]
    car_colors = ['r' for i in range(len(svo))]
    for i in range(len(svo)):
        if svo[i] < np.pi/8.0:
            car_colors[i] = 'red'
        elif np.pi/8 <= svo[i] <= 3*np.pi/8.0:
            car_colors[i] = 'purple'
        elif np.pi/8 <= svo[i] <= np.pi/2.0:
            car_colors[i] = 'blue'
else:
    car_colors = None



IMG_DIR = LOG_DIRECTORY + 'temp/'
print("Saving photos to %s..."%(IMG_DIR))

os.makedirs(IMG_DIR, exist_ok=True)    
for f in glob.glob(os.path.join(IMG_DIR, "*.png")):
    os.remove(f)
try:
    os.remove(VIDEO_OUT_PATH)
except OSError:
    pass

cmplot.plot_cars(world, response_vehicle, xamb, [xothers[i,:,:] for i in range(xothers.shape[0])],
                IMG_DIR, shape, camera_speed, None, car_colors, n_workers)              

outfile = cmplot.animate(IMG_DIR, VIDEO_OUT_PATH, fps)

vid_directory = '/home/nbuckman/mpc-vids/'
cmd = 'cp %s %s'%(VIDEO_OUT_PATH, vid_directory)
os.system(cmd)


Saving photos to /home/nbuckman/mpc_results/02_17_txe/61fc-3c7g-20210218-142624/61fc-3c7g-20210218-142624_00023/temp/...


0

In [8]:
from IPython.display import Video

Video(outfile, embed=True, width=1024)

In [9]:
outfile

'/home/nbuckman/mpc_results/02_17_txe/61fc-3c7g-20210218-142624/61fc-3c7g-20210218-142624_00023/61fc-3c7g-20210218-142624_00023.mp4'